In [32]:
import numpy as np
import scipy.special as sp
import random
from matplotlib import pyplot as plt
from itertools import permutations

$z$ is a list that contains the solutions to the Bethe equations:

$z_i^L = -\prod_{j=1}^N\left( -\frac{1 + z_i z_j - \Delta z_i}{1 + z_i z_j - \Delta z_j}\right), \quad i =1, 2, \dots, N$

So, $z  = [z_1, z_2, \dots, z_N]$

In [381]:
def M(z, i, j, d):
    return d*( 1 + z[i]*z[i] - d*z[i] )/((1 + z[i]*z[j] - d*z[j])*(1 + z[i]*z[j] - d*z[i]))

The following function gives the matrix for the coefficients

In [378]:
def Cmat(z, d, L):
    n = len(z)
    diag =0
    mat = [[ - M(z, i, j, d) for i in range(n)] for j in range(n) ]
    for i in range(n):
        diag = sum(mat[i]) +M(z, i, i, d)
        mat[i][i] = -diag + (L/z[i])
    return(mat)

In [157]:
def Coeff(y, z, d, L):
    n = len(z)
    terms = [ z[i]**(y[i]+1) for i in range(n)] 
    coeff = np.prod(terms)
    coeff = coeff*np.linalg.det(Cmat(z, d, L))
    coeff = 1/coeff
    return coeff

In [209]:
z = [1,2,3]
y = [1,2,3]
d = 1
L = 2
Coeff(y, z, d, L)

0.00016325464459463873

In [159]:
Cmat([1,2,3], 1, 10)

[[10.833333333333334, -0.5, -0.3333333333333333],
 [-1.5, 6.65, -0.15],
 [-2.3333333333333335, -0.35, 6.0166666666666675]]

In [160]:
list(permutations(range(3)))

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

In [344]:
def inversions(s):
    n = len(s)
    inv = []
    for i in range(n):
        for j in range(i+1, n):
            if s[i] > s[j]:
                inv.append((s[i],s[j]))
    return inv

In [345]:
inversions((0,2,1))

[(2, 1)]

In [366]:
def A(z, s, d):
    n = len(z)
    inv = inversions(s)
    terms = [ -(1 + z[i[0]]*z[i[1]] - d*z[i[0]])/(1 + z[i[0]]*z[i[1]] - d*z[i[1]]) for i in inv]
    return np.prod(terms)

In [367]:
z = [1,2]
s = (1,0)
d=4
A(z, s, d)

-5.0

In [332]:
def eigenfun(x, z, d):
    n = len(z)
    per = list(permutations(range(n)))
    final_term=0
    term = 0
    for s in per:
        term = A(z, s,d)
        for i in range(n):
            term = term*(z[s[i]]**(x[i]))
        final_term = final_term + term
    return final_term

# Solutions to the Bethe equations

In [372]:
def BE(z,i,d):
    n = len(z)
    term=1
    for j in range(n):
        term= term*(-1)*((1 + z[i]*z[j] - d*z[i])/(1 + z[i]*z[j] - d*z[j]))
    return -term

In [167]:
BE((1,2), 1, 0)

-1.0

In [168]:
def seq_update(z,d,L):
    n = len(z)
    w = []
    for i in range(n):
        pol = [1]
        pol.extend([0]*(L-1))
        pol.append(- BE(z, i, d))
        roots = np.roots(pol)
        roots = [x-z[i] for x in roots]
        w.append(min(roots, key=np.abs) + z[i])
    return w

In [169]:
def BE_sol_initial(I,L):
    n = len(I)
    pol = [1]
    pol.extend([0]*(L-1))
    pol.append((-1)**n)
    roots = np.roots(pol)
    sol =[]
    for i in I:
        sol.append(roots[i])
    return sol

In [170]:
BE_sol_initial((1,2), 4)

[(-0.7071067811865477-0.7071067811865476j),
 (0.7071067811865471+0.7071067811865469j)]

In [171]:
def tuple_combinations(N,L):
    def helper(current_tuple, index):
        
        if index == N:
            result.append(current_tuple)
            return
        if index ==0:
            for i in range(L):
                helper(current_tuple + (i,), index+1)
        else:
            for i in range(current_tuple[-1]+1, L):
                helper(current_tuple + (i,), index+1) 
    result = []
    helper(tuple(), 0)
    return result

In [334]:
N=3
L=10
print(tuple_combinations(N,L))

[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 2, 8), (0, 2, 9), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 9), (0, 4, 5), (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 4, 9), (0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 5, 9), (0, 6, 7), (0, 6, 8), (0, 6, 9), (0, 7, 8), (0, 7, 9), (0, 8, 9), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 2, 7), (1, 2, 8), (1, 2, 9), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 3, 7), (1, 3, 8), (1, 3, 9), (1, 4, 5), (1, 4, 6), (1, 4, 7), (1, 4, 8), (1, 4, 9), (1, 5, 6), (1, 5, 7), (1, 5, 8), (1, 5, 9), (1, 6, 7), (1, 6, 8), (1, 6, 9), (1, 7, 8), (1, 7, 9), (1, 8, 9), (2, 3, 4), (2, 3, 5), (2, 3, 6), (2, 3, 7), (2, 3, 8), (2, 3, 9), (2, 4, 5), (2, 4, 6), (2, 4, 7), (2, 4, 8), (2, 4, 9), (2, 5, 6), (2, 5, 7), (2, 5, 8), (2, 5, 9), (2, 6, 7), (2, 6, 8), (2, 6, 9), (2, 7, 8), (2, 7, 9), (2, 8, 9), (3, 4, 5), (3, 4, 6), (3, 4, 7), (3, 4, 8), (3, 4, 9), (3, 5, 6)

In [230]:
N=2
L=3
config = []
all_config = tuple_combinations(N,L)
for z in all_config:
    config.extend(list(permutations(z)))
print(config)

[(0, 1), (1, 0), (0, 2), (2, 0), (1, 2), (2, 1)]


In [227]:
if 1 in (0,1):
    print(1)

1


In [173]:
def BE_all_sol(N,L,d,updates):
    all_tuples = tuple_combinations(N,L)
    sol = []
    for I in all_tuples:
        initial = BE_sol_initial(I,L)
        for k in range(updates):
            initial = seq_update(initial, d, L)
        sol.append(initial)
    return sol

In [174]:
BE_all_sol(3,10, 0.1, 10)

[[(-1.0000000000000009-3.3306690738754696e-16j),
  (-0.8140582944453838+0.5807831723153436j),
  (-0.8140582944453847-0.5807831723153429j)],
 [(-0.9999513064075638+0.009868374425775372j),
  (-0.8080325126113613+0.5891378943532432j),
  (-0.3199727973554893+0.9474267301234975j)],
 [(-0.999992978231423-0.0037474641892302207j),
  (-0.8155552012954254+0.5786792839215686j),
  (-0.3232074690736634-0.9463281312182338j)],
 [(-0.999874485424833+0.01584340229951997j),
  (-0.8014054708658207+0.5981214519362545j),
  (0.2816168524811098+0.9595269399024886j)],
 [(-0.99995264504286-0.009731786669809228j),
  (-0.8174296123037114+0.5760284966292948j),
  (0.2859240465018509-0.9582522839169273j)],
 [(-0.8317166037830199-0.5552004061522635j),
  (-0.8317166037840173+0.5552004061507714j),
  (1.0000000000000002-1.7930434914603653e-12j)],
 [(-0.9996276213296389+0.027287701897530714j),
  (-0.7297940884983238+0.6836670157268779j),
  (0.7108666270236855+0.7033268362461184j)],
 [(-0.9996940953606196-0.0247328870355

# Checking initial conditions

In [400]:
N=4
L=7
d=0.01
ord_sol = BE_all_sol(N,L, d, 10)
all_sol =[]
for z in ord_sol:
    all_sol.extend(list(permutations(z)))
config = []
all_config = tuple_combinations(N,L)
#print(all_config)
trans_mat = [ [ np.abs(sum([Coeff(y, z, d, L)*eigenfun(x, z, d) for z in all_sol])) for y in all_config] for x in all_config]
print(np.matrix(trans_mat))

[[1.00000000e+00 4.44982004e-16 4.03185127e-16 ... 1.29469871e-15
  1.10974649e-15 1.31840732e-15]
 [4.51077841e-16 1.00000000e+00 4.28788472e-16 ... 4.02252989e-16
  4.78815497e-16 1.13181547e-15]
 [5.32137913e-16 7.81865741e-16 1.00000000e+00 ... 2.98675502e-16
  1.85450175e-16 1.01793617e-15]
 ...
 [8.40923092e-16 3.86742276e-16 4.98450084e-17 ... 1.00000000e+00
  4.14133441e-16 6.99758831e-16]
 [7.83018495e-16 5.14453669e-16 3.64955528e-16 ... 3.48820461e-16
  1.00000000e+00 3.62422580e-16]
 [7.34967466e-16 6.66783230e-16 1.31966027e-15 ... 7.57674845e-16
  2.69954584e-16 1.00000000e+00]]


In [292]:
N=2
L=3
d=0.01
ord_sol = BE_all_sol(N,L, d, 100)
ord_sol[2][1]**L - BE(ord_sol[2], 1, d)

(-5.551115123125783e-16+3.5214886562329184e-16j)

In [ ]:
N=2
L=3
d=0.01
ord_sol = BE_all_sol(N,L, d, 10)
all_sol =[]
for z in ord_sol:
    all_sol.extend(list(permutations(z)))
config = []
all_config = tuple_combinations(N,L)
#print(all_config)
trans_mat = [ [ np.abs(sum([Coeff(y, z, d, L)*eigenfun(x, z, d) for z in all_sol])) for y in all_config] for x in all_config]
print(np.matrix(trans_mat))

In [382]:
N=2
L=5
d=0.1
y=[-1,-1]
ord_sol = BE_all_sol(N,L, d, 10)
all_sol =[]
for z in ord_sol:
    all_sol.extend(list(permutations(z)))
[Coeff(y, z, d, L) for z in all_sol]

[(0.012604368615870352-0.03879225778555262j),
 (0.012604368615870356-0.03879225778555262j),
 (0.012604368615870352+0.03879225778555262j),
 (0.012604368615870356+0.03879225778555262j),
 (-0.033161908385960226-0.024093536752217697j),
 (-0.03316190838596021-0.024093536752217683j),
 (-0.033161908385960226+0.024093536752217697j),
 (-0.03316190838596021+0.024093536752217683j),
 (0.04026623367940392+0j),
 (0.04026623367940392+0j),
 (-0.030604906692462387+0.0222357662777098j),
 (-0.03060490669246238+0.022235766277709804j),
 (0.012275661153956936+0.03778060024147512j),
 (0.012275661153956936+0.03778060024147512j),
 (0.012275661153956936-0.03778060024147512j),
 (0.012275661153956936-0.03778060024147512j),
 (-0.030604906692462387-0.0222357662777098j),
 (-0.03060490669246238-0.022235766277709804j),
 (0.03934541680603301+0j),
 (0.03934541680603301+0j)]

In [373]:
N=2
L=5
d=0.1
y=[-1,-1]
ord_sol = BE_all_sol(N,L, d, 10)
all_sol =[]
for z in ord_sol:
    all_sol.extend(list(permutations(z)))
print(all_sol)

[((-0.9999004932239451-0.014106865367932042j), (-0.29556981885742095+0.9553211408634221j)), ((-0.29556981885742095+0.9553211408634221j), (-0.9999004932239451-0.014106865367932042j)), ((-0.9999004932239451+0.014106865367932042j), (-0.29556981885742095-0.9553211408634221j)), ((-0.29556981885742095-0.9553211408634221j), (-0.9999004932239451+0.014106865367932042j)), ((-0.9982161702461375-0.059703244963180246j), (0.8426665326923637+0.5384358036760089j)), ((0.8426665326923637+0.5384358036760089j), (-0.9982161702461375-0.059703244963180246j)), ((-0.9982161702461375+0.059703244963180246j), (0.8426665326923637-0.5384358036760089j)), ((0.8426665326923637-0.5384358036760089j), (-0.9982161702461375+0.059703244963180246j)), ((-0.2910397456371137+0.9567109628615555j), (-0.2910397456371137-0.9567109628615555j)), ((-0.2910397456371137-0.9567109628615555j), (-0.2910397456371137+0.9567109628615555j)), ((-0.26950246119552274+0.9629997006279674j), (0.7840690931242393+0.6206735512387601j)), ((0.78406909312